In [9]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.structural import UnobservedComponents
import numpy as np
from filterpy.kalman import KalmanFilter
import matplotlib.pyplot as plt

In [10]:
df = pd.read_csv(r"D:\TimeSeries\BTTH1\test.csv")
df = df.loc[(df['Publication_Day']=='Wednesday')]
df.head()

id       Podcast_Name Episode_Title  Episode_Length_minutes  \
4   750004       Life Lessons    Episode 50                   72.32   
15  750015         Tech Talks     Episode 4                   87.02   
16  750016        Style Guide     Episode 1                     NaN   
22  750022  Business Insights    Episode 78                  102.69   
34  750034         Laugh Line    Episode 81                   33.85   

         Genre  Host_Popularity_percentage Publication_Day Publication_Time  \
4    Lifestyle                       58.10       Wednesday          Morning   
15  Technology                       86.21       Wednesday        Afternoon   
16   Lifestyle                       63.87       Wednesday          Evening   
22    Business                       25.40       Wednesday            Night   
34      Comedy                       29.35       Wednesday            Night   

    Guest_Popularity_percentage  Number_of_Ads Episode_Sentiment  
4                         11.30            2.0           Neutral  
15                        67.42            3.0          Negative  
16                        86.45            0.0          Positive  
22                        45.51            0.0          Positive  
34                        41.27            0.0          Positive

In [11]:
values = df['Episode_Length_minutes'].values
df.sort_values(by="id", inplace=True)
ts = df["Episode_Length_minutes"].reset_index(drop=True)

In [12]:
# Xử lý giá trị thiếu
cols_mean_fill = ['Episode_Length_minutes', 'Host_Popularity_percentage', 'Number_of_Ads']
for col in cols_mean_fill:
    if col in df.columns:
        df[col].fillna(df[col].mean(), inplace=True)

if 'Guest_Popularity_percentage' in df.columns:
    df['Guest_Popularity_percentage'].fillna(0, inplace=True)

C:\Users\May\AppData\Local\Temp\ipykernel_20488\3034424329.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)
C:\Users\May\AppData\Local\Temp\ipykernel_20488\3034424329.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, wh

In [13]:
# One-Hot Encoding for string columns
string_cols = ['Podcast_Name', 'Episode_Title', 'Genre', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment']
df = pd.get_dummies(df, columns=string_cols)

In [14]:
# Mô hình 1: Chỉ có mức trung bình (local level)
model1 = UnobservedComponents(ts, level='local level')
result1 = model1.fit()

In [15]:
# Mô hình 2: Mức trung bình + xu hướng tuyến tính (local linear trend)
model2 = UnobservedComponents(ts, level='local linear trend')
result2 = model2.fit()

In [16]:
# Mô hình 3: Thêm thành phần chu kỳ (cycle)
model3 = UnobservedComponents(ts, level='local level', cycle=True)
result3 = model3.fit()


In [17]:
# Vẽ kết quả
plt.figure(figsize=(10, 6))

plt.subplot(3, 1, 1)
plt.plot(ts, label="Dữ liệu gốc", alpha=0.4)
plt.plot(result1.filtered_state[0], label="Mô hình 1: Local Level", color="red")
plt.legend()

plt.subplot(3, 1, 2)
plt.plot(ts, label="Dữ liệu gốc", alpha=0.4)
plt.plot(result2.filtered_state[0], label="Mô hình 2: Linear Trend", color="green")
plt.legend()

plt.subplot(3, 1, 3)
plt.plot(ts, label="Dữ liệu gốc", alpha=0.4)
plt.plot(result3.filtered_state[0], label="Mô hình 3: Chu kỳ", color="blue")
plt.legend()

plt.tight_layout()
plt.suptitle("Phân tích chuỗi thời gian bằng Kalman Filter (Publication_Day = Wednesday)", y=1.02)
plt.show()